# SQL AND Python 
This notebook is created to save and apply the techniques to connect to MySQL database through Python.
The notes are taken from the incredible [online Tutorial](https://www.mysqltutorial.org/python-mysql/) 

## connect to a database
[first tutorial](https://www.mysqltutorial.org/getting-started-mysql-python-connector/) 

In [2]:
# let's first install the mysql connector
! pip install mysql-connector-python

In [3]:
# now let's connect
from mysql import connector

# on example of connection: 
connector.connect(host='localhost', database='sql_zoo', user='ayhem_vscode', password='Imadeyoumy97*')

The 2nd [tutorial](https://www.mysqltutorial.org/python-connecting-mysql-databases/) provides a SQL script to create 3 different tables for illustration and practice purposes.

In [4]:
# importing required modules
import os
import random
from zipfile import ZipFile
# current directory
CURRENT_DIR = os.getcwd() 
file_name = "python_mysql.zip"
file_name_no_zip = "mysql_python"
loc = os.path.join(CURRENT_DIR, file_name)
# opening the zip file in READ mode

if not os.path.isdir(os.path.join(CURRENT_DIR, file_name_no_zip)):
    with ZipFile(loc, 'r') as zip_ref:
        zip_ref.extractall()
        print('Done!')

Done!


The following commands (throug the command line) can be used to create the new database
1. mysql -u root -p
2. type the user name as well as the password
3. create database python_mysql; 
4. use python_mysql
5. source **absolute_file_path** 
6. to double-check: show tables; (displays the tables in the current database)

In [6]:
from mysql.connector import Error

# let's wrap the connection procedure in a function
def connect(host='localhost', database='python_mysql', user='ayhem_vscode', password='Imadeyoumy97*'):
    with connector.connect(host=host, database=database, user=user, password=password) as connection:
        try:
            if connection.is_connected():
                print("the connection is established!!")             
        except Error as e:
            print(e)
    return connection

conn = connect()

the connection is established!!


In [21]:
from configparser import ConfigParser
# hardcoding the credential is not as safe and is error-prone.
# One more reliable way it to used configuration files
def read_db_config(config_file_path: str, section:str='mysql') -> dict:
    """
    This function reads configuration file, parse the credentials and return them in a dictionary-like object.
    : param config_file_path: the path to the configuration file
    : param section: the section to consider from the configuration file
    : return  a dictionary with the connection's parameters
    """
    # initialize a parser object
    parser = ConfigParser()
    parser.read(config_file_path)
    
    # check if the section set in the argument is indeed in the configuration file (read by the parser)
    try:
        if parser.has_section(section):
            # extract the items 
            items = parser.items(section)
            # convert them to a dict
            return dict((item[0], item[1]) for item in items)

    except Error as e:
        print(e)

    # the reuslt will be of None type in case the section is not present in the configuration file
    

# let's see how the function works
# let's have a function that creates a configuration file given hard-coded credentials
def write_db_config(host: str, database: str, user: str, password:str, config_file_path:str=None, section:str='mysql'):
    if config_file_path is None:
        config_file_path = os.path.join(os.getcwd(), f'{user}_host_database')
    
    # let's first check if the file ends with the appropriate extension and add if necessary
    if not config_file_path.endswith('.ini'):
        config_file_path += '.ini'

    params = {"host": host, "database":database, "user": user, "password": password}
    # create the file
    with open(config_file_path, 'w') as f:
        f.write(f'[{section}]\n')
        for param, param_value in params.items():
            f.write(f'{param}= {param_value}\n')
    return config_file_path

from mysql.connector import MySQLConnection

# let's introduce a function that will connect via config files
def connect_config(config_file_path, section='mysql'):
    # extract the credential from the configuration file
    db_config = read_db_config(config_file_path, section=section)
    try:    
        with MySQLConnection(**db_config) as conn:
            if conn.is_connected():
                print('Connection established.')
            else:
                print('Connection failed.')

        return conn
    except Error as e:
        print(e)

In [24]:
# let's the functions to test

config_file_name = write_db_config(host='localhost', database='python_mysql', user='ayhem_vscode', password='Imadeyoumy97*')
connection = connect_config(config_file_name)


Connection established.
